<a href="https://colab.research.google.com/github/miamvmian/MLHW/blob/main/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/miamvmian/Sk-ML-FNO-Project-2024.git

Cloning into 'Sk-ML-FNO-Project-2024'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 37 (delta 11), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (37/37), 20.69 MiB | 13.69 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [3]:
import torch
import os
from pathlib import PurePath


cwd = os.getcwd()
p = PurePath(cwd)
flist = os.listdir()

mainfolder = 'Sk-ML-FNO-Project-2024'

# os.chdir('Sk-ML-FNO-Project-2024')
if p.parts[-1] != mainfolder and mainfolder in flist:
  os.chdir(mainfolder)
print(f"Current Folder is: {os.getcwd()}")

# from google.colab import drive
# drive.mount('/content/drive')

Current Folder is: /content/Sk-ML-FNO-Project-2024


In [6]:
from Model import FNO1d
modes = 16
width = 64
model = FNO1d(modes,width)
state_dict = torch.load("fno-1d-10layers.pth",map_location=torch.device('cpu'))
model.load_state_dict(state_dict)